# Using eBFE Models: North Galveston Bay HMS + RAS Integration

This notebook demonstrates working with compound eBFE/BLE models that include both HEC-HMS hydrologic and HEC-RAS hydraulic models.

## The Problem: eBFE Models Are Broken

**FEMA's eBFE models are fundamentally broken and unusable without significant manual effort:**

1. **HMS and RAS Mixed**: Hydrologic and hydraulic models in same archive with no clear separation
2. **Nested 6.1 GB Zip**: RAS model buried in nested zip requiring manual extraction (command-line tools fail)
3. **Absolute DSS Paths**: HMS DSS outputs use absolute paths → HEC-RAS can't find them
4. **Terrain Separated**: Terrain outside project folder → Model won't run
5. **Output/ Separated**: Pre-run results inaccessible

**Manual Fix Time**: 45-90 minutes per model (HMS extraction, RAS extraction, path corrections, folder moves)

**With RasEbfeModels**: Automatic HMS/RAS separation, path corrections, folder integration ✓

## Our Solution: Automated Organization + Path Fixing

**RasEbfeModels.organize_north_galveston_bay() automatically**:
1. Separates HMS Model/ from RAS Model/
2. Organizes 7 storm frequencies + sensitivity analysis
3. Corrects DSS paths to relative references  
4. Moves Terrain/ and Output/ when RAS extracted
5. Creates runnable models (or clear extraction instructions)

## Model Characteristics

- **Pattern 4**: Compound HMS + RAS archive (most complex)
- **Size**: 8.2 GB
- **HMS**: NorthGalvestonBay.hms with 7 storm frequencies + sensitivity
- **RAS**: 2D coastal hydraulic model (nested in RAS_Submittal.zip)
- **Type**: Coastal flood analysis (hurricane surge)

## What You'll Learn

1. Organize compound HMS + RAS model (most complex pattern)
2. Understand the critical fixes applied automatically
3. Work with HMS hydrologic model (multiple storm frequencies)
4. Handle large nested RAS extraction (6.1 GB, manual required)
5. Understand HMS → RAS workflow
6. Validate DSS time series data

## Prerequisites

**Downloads Required**:
1. North Galveston Bay Models.zip (8.2 GB) - HMS + nested RAS
2. North Galveston Bay Documents.zip (20 MB) - BLE reports

**Note**: This is Pattern 4 - most complex pattern with both hydrologic and hydraulic models.

In [ ]:
from pathlib import Path
import sys

# Add parent directory to path for development
try:
    from ras_commander import init_ras_project, RasCmdr
except ImportError:
    sys.path.insert(0, str(Path.cwd().parent))
    from ras_commander import init_ras_project, RasCmdr

import pandas as pd

## Step 1: Organize Downloaded Model

Use `RasEbfeModels.organize_north_galveston_bay()` to organize HMS and RAS content into separate folders.

In [ ]:
# Import eBFE model organization function
from ras_commander.ebfe_models import RasEbfeModels

# Set paths
downloaded_folder = Path(r"D:\Ras-Commander_BulkData\eBFE\Harris_County\12040203_NorthGalvestonBay_Models_extracted")
organized_folder = Path(r"D:\Ras-Commander_BulkData\eBFE\Organized\NorthGalvestonBay_12040203")

# Organize (HMS will be ready, RAS requires manual extraction)
if not organized_folder.exists() or not (organized_folder / "agent" / "model_log.md").exists():
    print("Organizing North Galveston Bay model...")
    print("Note: RAS model (6.1 GB nested zip) requires manual extraction\n")
    
    organized_folder = RasEbfeModels.organize_north_galveston_bay(
        downloaded_folder,
        organized_folder,
        extract_ras_nested=False,  # Manual extraction recommended
        validate_dss=True
    )
else:
    print(f"Model already organized at: {organized_folder}")

print(f"\n✓ Organized model location: {organized_folder}")

## Step 2: Explore HMS Model

Pattern 4 includes HEC-HMS hydrologic model with multiple storm frequencies.

In [ ]:
# HMS model location
hms_folder = organized_folder / "HMS Model" / "NorthGalvestonBay"
hms_project = hms_folder / "NorthGalvestonBay.hms"

if hms_project.exists():
    print(f"HMS Project: {hms_project.name}")
    print(f"Location: {hms_folder}\n")
    
    # List storm frequencies
    dss_files = list(hms_folder.glob('*.dss'))
    print(f"Storm Frequencies ({len(dss_files)} DSS files):")
    for dss in sorted(dss_files):
        size_mb = dss.stat().st_size / 1e6
        print(f"  - {dss.stem}: {size_mb:.1f} MB")
    
    # List basin models
    basin_files = list(hms_folder.glob('*.basin'))
    print(f"\nBasin Models ({len(basin_files)}):")
    for basin in sorted(basin_files):
        print(f"  - {basin.name}")
        
else:
    print("⚠️ HMS project not found - check organization")

## Step 3: Explore HMS DSS Time Series

HMS models generate streamflow hydrographs in DSS format that feed into HEC-RAS.

In [ ]:
from ras_commander.dss import RasDss

# Pick one storm frequency to explore
dss_file = hms_folder / "100yr.dss"

if dss_file.exists():
    print(f"Exploring: {dss_file.name}\n")
    
    # Get catalog of all pathnames
    catalog = RasDss.get_catalog(dss_file)
    print(f"Total pathnames: {len(catalog)}")
    
    # Show first 10 pathnames
    print(f"\nFirst 10 pathnames:")
    for i, pathname in enumerate(catalog['pathname'][:10]):
        print(f"  {i+1}. {pathname}")
    
    if len(catalog) > 10:
        print(f"  ... and {len(catalog) - 10} more")
    
    # Validate DSS pathnames
    print(f"\nValidating DSS pathnames...")
    invalid_count = 0
    for pathname in catalog['pathname']:
        result = RasDss.check_pathname(dss_file, pathname)
        if not result.is_valid:
            invalid_count += 1
    
    if invalid_count == 0:
        print(f"  ✓ All {len(catalog)} pathnames valid")
    else:
        print(f"  ⚠️ {invalid_count} invalid pathname(s) found")
else:
    print(f"⚠️ {dss_file.name} not found")

## Step 4: Check RAS Model Status

Pattern 4 RAS model is in a nested 6.1 GB zip. Check extraction status.

In [ ]:
ras_folder = organized_folder / "RAS Model"
extraction_readme = ras_folder / "README_EXTRACTION_NEEDED.txt"

# Check if RAS has been extracted
ras_prj_files = list(ras_folder.glob('**/*.prj'))
# Filter out shapefile .prj
ras_prj_files = [p for p in ras_prj_files if 'Shp' not in str(p) and 'Features' not in str(p)]

if ras_prj_files:
    print("✓ RAS model extracted successfully")
    print(f"  Found: {ras_prj_files[0]}")
    ras_extracted = True
elif extraction_readme.exists():
    print("⚠️ RAS model requires manual extraction")
    print(f"\nInstructions: {extraction_readme}")
    print("\nExtraction Steps:")
    print(extraction_readme.read_text())
    ras_extracted = False
else:
    print("⚠️ RAS model status unknown")
    ras_extracted = False

## Step 5: Initialize RAS Model (If Extracted)

If RAS_Submittal.zip has been manually extracted, initialize with ras-commander.

In [ ]:
if ras_extracted:
    print("Initializing RAS model...")
    
    # Initialize project
    ras_project = ras_prj_files[0].parent
    ras = init_ras_project(ras_project, "6.5")  # Version TBD from files
    
    print(f"✓ Project initialized: {ras.prj_file}")
    print(f"\nPlans found: {len(ras.plan_df)}")
    print(ras.plan_df[['plan_number', 'plan_title']].to_string())
    
    # Validate DSS files in RAS model
    print("\nValidating RAS DSS files...")
    ras_dss_files = list(ras_project.glob('**/*.dss'))
    print(f"  Found {len(ras_dss_files)} DSS file(s)")
    
    for dss in ras_dss_files:
        catalog = RasDss.get_catalog(dss)
        print(f"  {dss.name}: {len(catalog)} pathname(s)")
else:
    print("⚠️ RAS model not extracted - complete manual extraction first")
    print("\nAfter extraction, re-run this cell to initialize RAS model")

## Step 6: Understanding HMS → RAS Workflow

Pattern 4 models show the typical HMS → RAS workflow:

1. **HEC-HMS**: Generates streamflow hydrographs from precipitation
2. **DSS Transfer**: HMS results saved to DSS files
3. **HEC-RAS**: Uses DSS hydrographs as boundary conditions
4. **2D Routing**: Computes inundation depths and extents

In [ ]:
print("HMS → RAS Workflow for North Galveston Bay:\n")
print("Step 1: HEC-HMS Hydrologic Modeling")
print("  - Input: Design storm precipitation (10yr, 25yr, 50yr, 100yr, 500yr)")
print("  - Process: Basin rainfall-runoff transformation")
print("  - Output: Streamflow hydrographs → DSS files")
print(f"  - Location: {hms_folder}")
print()
print("Step 2: DSS Time Series")
print("  - HMS results stored in .dss files")
print("  - Multiple storm frequencies (7 total)")
print("  - Sensitivity variants: 100yr_Min, 100yr_Plus")
print()
print("Step 3: HEC-RAS Hydraulic Modeling")
print("  - Input: HMS hydrographs from DSS + coastal boundary conditions")
print("  - Process: 2D shallow water equations")
print("  - Output: Inundation maps, depths, velocities")
if ras_extracted:
    print(f"  - Location: {ras_project}")
else:
    print(f"  - Status: Pending manual extraction")

## Step 7: Review Documentation

Pattern 4 models include comprehensive BLE reports and metadata.

In [ ]:
docs_folder = organized_folder / "Documentation"
doc_files = list(docs_folder.glob('*'))

print(f"Documentation Files ({len(doc_files)}):")
print("=" * 80)
for doc in sorted(doc_files):
    size_mb = doc.stat().st_size / 1e6
    print(f"  - {doc.name}: {size_mb:.2f} MB")

# Read model inventory if available
inventory = docs_folder / "2D_Model_Inventory.xlsx"
if inventory.exists():
    print(f"\nReading model inventory...")
    df = pd.read_excel(inventory, sheet_name=0, nrows=10)
    print(df.to_string())

## Step 8: Check Agent Work Log

Every organized model has agent/model_log.md documenting the organization process.

In [ ]:
model_log = organized_folder / "agent" / "model_log.md"

if model_log.exists():
    print(f"Agent Work Log: {model_log}\n")
    print("Preview (first 30 lines):")
    print("=" * 80)
    log_content = model_log.read_text()
    print('\n'.join(log_content.split('\n')[:30]))
    print("=" * 80)
else:
    print("⚠️ Agent work log not found")

## Optional: Extract Results from RAS Model

**Requires**: Manual extraction of RAS_Submittal.zip first

After extracting the RAS model, you can initialize and extract results.

In [ ]:
if ras_extracted:
    print("Extracting RAS results...\n")
    
    # Check for HDF result files
    hdf_files = list(ras_project.glob('**/*.p*.hdf'))
    print(f"HDF result files: {len(hdf_files)}")
    
    if hdf_files:
        # Extract from first HDF file
        from ras_commander.hdf import HdfResultsPlan
        
        hdf = HdfResultsPlan(hdf_files[0])
        wse = hdf.get_wse(time_index=-1)
        
        print(f"\nResults from {hdf_files[0].name}:")
        print(f"  WSE count: {len(wse)}")
        print(f"  WSE range: {wse.min():.2f} to {wse.max():.2f} ft")
    else:
        print("  No HDF files found - may need to run compute test")
else:
    print("⚠️ RAS model not extracted")
    print("\nTo extract:")
    print("  1. See RAS Model/README_EXTRACTION_NEEDED.txt")
    print("  2. Extract RAS_Submittal.zip (6.1 GB) via Windows Explorer")
    print("  3. Re-run this notebook")

## Summary

This notebook demonstrated:

1. ✓ **Pattern 4 Organization**: Compound HMS + RAS model organization
2. ✓ **HMS Content**: 7 storm frequencies + sensitivity analysis
3. ✓ **DSS Exploration**: HMS-generated streamflow hydrographs
4. ✓ **Documentation**: BLE reports and metadata
5. ✓ **Agent Log**: Complete organization documentation
6. ⚠️ **RAS Integration**: Requires manual extraction (large nested zip)

**Pattern 4 Characteristics**:
- Most complex pattern (HMS + RAS)
- Multiple storm frequencies (10yr through 500yr)
- Sensitivity analysis (100yr_Min, 100yr_Plus)
- Large nested RAS zip may require manual extraction
- Coastal flood analysis (hurricane surge)

**HMS → RAS Workflow**:
```
HEC-HMS (precipitation → streamflow)
   ↓ DSS files
HEC-RAS (streamflow → inundation)
```

**Next Steps**:
- Extract RAS_Submittal.zip manually (6.1 GB)
- Initialize RAS model with ras-commander
- Run compute test to validate HMS → RAS workflow
- Extract and visualize coastal inundation results